# Метрики графа: статьи (Article)

Ноутбук для наглядного анализа метрик **только по статьям** (узлы = статьи энциклопедии).  
Связи с авторами, публикациями и формулами учитываются как рёбра графа; **входящие ссылки между статьями** в онтологии идут на **Term**, а не на Article — это учтено в расчётах.

**Два способа получения данных:**
1. **Python + rdflib**: загрузка TTL-файлов из папки `graph/`.
2. **SPARQL**: запросы к GraphDB (при необходимости — выполнение из Python через `SPARQLWrapper`).

## 1. Загрузка данных

Подгружаем граф из TTL (локально) и при необходимости подключаемся к GraphDB для SPARQL.

In [56]:
import sys
from pathlib import Path
from collections import defaultdict

import pandas as pd
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib.namespace import RDF, RDFS

# Локальный модуль загрузки
sys.path.insert(0, str(Path.cwd()))
from graph_loader import load_graph_from_ttl, DEFAULT_GRAPH_DIR, SG, FME, ME

# Для метрик центральности
import networkx as nx

# Загрузка графа из TTL
g = load_graph_from_ttl(DEFAULT_GRAPH_DIR)
print(f"Загружено триплов: {len(g)}")

Загружено триплов: 480055


### Собираем только статьи и объектные свойства для статей

Узлы — все объекты типа `sg:Article`. Рёбра считаем по объектным свойствам (без datatype).  
**Важно:** входящие ссылки на статью со стороны других статей идут в онтологии в **Term** (предикат `sg:hasRelation`), поэтому входящая степень статьи учитывает и «ссылки на термин» этой статьи.

In [57]:
# Объектные свойства, исходящие из Article (для исходящей степени)
ARTICLE_OUT_PREDICATES = {
    # str(SG.hasAuthor),
    # str(SG.hasTerm),
    # str(SG.hasThesaurus),
    # str(SG.refersTo),
    # str(SG.hasFormula),
    str(SG.hasRelation),
}

# Предикаты, где Article — объект (входящая степень)
# hasArticle (Person -> Article), referredIn (Publication -> Article), usedIn (Formula -> Article)
ARTICLE_IN_PREDICATES = {
    # str(SG.hasArticle),   # Person hasArticle Article
    # str(SG.referredIn),   # Publication referredIn Article
    # str(SG.usedIn),       # Formula usedIn Article
}

# hasRelation: субъект Article, объект Term — входящие к статье считаем по Term
# (сколько раз какой-то Article ссылается на Term этой статьи)

In [58]:
# Все статьи
articles = list(g.subjects(RDF.type, SG.Article))
article_uri_to_label = {}
for a in articles:
    label = g.value(a, RDFS.label, None)
    if label is not None:
        article_uri_to_label[str(a)] = str(label)
    else:
        article_uri_to_label[str(a)] = str(a)

# Article -> её Term (по hasTerm; у статьи ровно один Term в нашей модели)
article_to_term = {}
for a, _, t in g.triples((None, SG.hasTerm, None)):
    if (a, RDF.type, SG.Article) in g:
        article_to_term[str(a)] = str(t)

# Term -> список статей (одно название может быть у нескольких статей)
term_to_articles = defaultdict(list)
for a, _, t in g.triples((None, SG.hasTerm, None)):
    if (a, RDF.type, SG.Article) in g:
        term_to_articles[str(t)].append(str(a))

print(f"Число статей (узлов): {len(articles)}")
print(f"Статей с термином: {len(article_to_term)}")

Число статей (узлов): 9758
Статей с термином: 9758


In [59]:
# Исходящая степень: (article, p, o) для p in ARTICLE_OUT_PREDICATES
out_degree = defaultdict(int)
for a in articles:
    sa = str(a)
    for p, o in g.predicate_objects(a):
        if str(p) in ARTICLE_OUT_PREDICATES:
            out_degree[sa] += 1

# Входящая степень: (s, p, article) для p in ARTICLE_IN_PREDICATES
# + входящие по Term: (other_article, hasRelation, term_this_article)
in_degree = defaultdict(int)
for s, p, o in g:
    if str(p) in ARTICLE_IN_PREDICATES and (o, RDF.type, SG.Article) in g:
        in_degree[str(o)] += 1

# Входящие на Term этой статьи (ссылки других статей на термин = входящие к статье)
for s, p, o in g.triples((None, SG.hasRelation, None)):
    if (s, RDF.type, SG.Article) in g:
        for art in term_to_articles.get(str(o), []):
            in_degree[art] += 1

# Общая степень = in + out
total_degree = {a: in_degree.get(a, 0) + out_degree.get(a, 0) for a in article_uri_to_label}

## 2. Общие метрики

- **Общее количество узлов** (статей)  
- **Общее количество связей**: считаем по объектным свойствам, инцидентным статьям (каждое ребро один раз — по исходящим из статей).

In [60]:
n_nodes = len(articles)
n_edges = sum(out_degree.values())  # каждое ребро от статьи учтено ровно раз
print("Общие метрики (статьи)")
print("  Узлов (статей):", n_nodes)
print("  Связей (рёбер):", n_edges)

Общие метрики (статьи)
  Узлов (статей): 9758
  Связей (рёбер): 57568


### Эквивалент через SPARQL (для GraphDB)

Ниже — те же показатели через SPARQL. Раскомментируйте и задайте `endpoint`, если используете GraphDB.

In [61]:
# SPARQL: количество статей и количество исходящих рёбер от статей
SPARQL_COUNTS = """
PREFIX sg: <https://scilib.ai/ontology/semantic-graph/>
SELECT (COUNT(DISTINCT ?article) AS ?nodes) (COUNT(*) AS ?edges)
WHERE {
  ?article a sg:Article .
  ?article ?p ?o .
  FILTER (?p IN (sg:hasAuthor, sg:hasTerm, sg:hasThesaurus, sg:refersTo, sg:hasFormula, sg:hasRelation))
}
"""
# from graph_loader import run_sparql
# rows = run_sparql(SPARQL_COUNTS, endpoint="http://localhost:7200/repositories/your_repo")
# if rows: print("Узлов:", rows[0].get("nodes"), "Связей:", rows[0].get("edges"))
print("SPARQL-запрос для подсчёта сохранён в переменной SPARQL_COUNTS")

SPARQL-запрос для подсчёта сохранён в переменной SPARQL_COUNTS


## 3. Распределение узлов по тезаурусам

Статьи связаны с тезаурусом через `sg:hasThesaurus` (значения `fme:` или `me:`).

In [62]:
thesaurus_count = defaultdict(int)
for a, _, th in g.triples((None, SG.hasThesaurus, None)):
    if (a, RDF.type, SG.Article) in g:
        uri = str(th)
        if "fme" in uri:
            thesaurus_count["FME (Энциклопедия мат. физики)"] += 1
        elif "me" in uri:
            thesaurus_count["ME (Математическая энциклопедия)"] += 1
        else:
            thesaurus_count[uri] += 1

df_thesaurus = pd.DataFrame([
    {"Тезаурус": k, "Количество статей": v} for k, v in sorted(thesaurus_count.items())
])
df_thesaurus

,Тезаурус,Количество статей
0,FME (Энциклопедия мат. физики),3586
1,ME (Математическая энциклопедия),6172


## 4. Средняя степень узлов

In [63]:
avg_degree = sum(total_degree.values()) / n_nodes if n_nodes else 0
avg_in = sum(in_degree.values()) / n_nodes if n_nodes else 0
avg_out = sum(out_degree.values()) / n_nodes if n_nodes else 0
print("Средняя степень (общая):", round(avg_degree, 4))
print("Средняя входящая степень:", round(avg_in, 4))
print("Средняя исходящая степень:", round(avg_out, 4))

Средняя степень (общая): 12.7533
Средняя входящая степень: 6.8538
Средняя исходящая степень: 5.8996


## 5. Топ-10 узлов по степени (с названиями)

Во всех таблицах — **названия статей** (rdfs:label).  
**Напоминание:** входящие связи между статьями идут в Term; при расчёте входящей степени мы учитываем ссылки на термин данной статьи.

In [64]:
def top10_df(article_scores, title="Степень"):
    items = sorted(article_scores.items(), key=lambda x: -x[1])[:10]
    return pd.DataFrame([
        {"Название статьи": article_uri_to_label.get(a, a), "URI": a, title: v}
        for a, v in items
    ])

print("Топ-10 по общей степени:")
display(top10_df(total_degree, "Общая степень"))

Топ-10 по общей степени:


,Название статьи,URI,Общая степень
0,ФУНКТОР,http://libmeta.ru/me/article/790_FUNKTO,1478
1,ТЕОРИЯ,http://libmeta.ru/me/article/381_TEORI,1425
2,МНОЖЕСТВО,http://libmeta.ru/me/article/733_MNOZhESTV,1207
3,ПРОСТРАНСТВО,http://libmeta.ru/me/article/748_PROSTRANSTV,1189
4,ПРОСТРАНСТВО,http://libmeta.ru/me/article/752_PROSTRANSTV,1189
5,ПЕРЕГО́РОДК,http://libmeta.ru/me/article/222_PEREGÓROD,886
6,ПРЕДСТАВЛЕНИЕ,http://libmeta.ru/me/article/3_PREDSTAVLENI,871
7,ПРОИЗВОДНАЯ,http://libmeta.ru/me/article/1045_PROIZVODNA,788
8,СИСТЕМА,http://libmeta.ru/me/article/350_SISTEM,771
9,ГРУПІІА,http://libmeta.ru/me/article/1207_GRUPІІ,730


In [65]:
print("Топ-10 по входящей степени (в т.ч. ссылки на Term статьи):")
display(top10_df(dict(in_degree), "Входящая степень"))

Топ-10 по входящей степени (в т.ч. ссылки на Term статьи):


,Название статьи,URI,Входящая степень
0,ФУНКТОР,http://libmeta.ru/me/article/790_FUNKTO,1463
1,ТЕОРИЯ,http://libmeta.ru/me/article/381_TEORI,1423
2,МНОЖЕСТВО,http://libmeta.ru/me/article/733_MNOZhESTV,1203
3,ПРОСТРАНСТВО,http://libmeta.ru/me/article/748_PROSTRANSTV,1182
4,ПРОСТРАНСТВО,http://libmeta.ru/me/article/752_PROSTRANSTV,1182
5,ПЕРЕГО́РОДК,http://libmeta.ru/me/article/222_PEREGÓROD,880
6,ПРЕДСТАВЛЕНИЕ,http://libmeta.ru/me/article/3_PREDSTAVLENI,853
7,ПРОИЗВОДНАЯ,http://libmeta.ru/me/article/1045_PROIZVODNA,784
8,СИСТЕМА,http://libmeta.ru/me/article/350_SISTEM,769
9,ГРУПІІА,http://libmeta.ru/me/article/1207_GRUPІІ,722


In [66]:
print("Топ-10 по исходящей степени:")
display(top10_df(dict(out_degree), "Исходящая степень"))

Топ-10 по исходящей степени:


,Название статьи,URI,Исходящая степень
0,ГОМЕОМОРФИЗМ,http://libmeta.ru/me/article/1116_GOMEOMORFIZ,75
1,АЛГЕБРА,http://libmeta.ru/me/article/94_ALGEBR,70
2,КОММУТАТИВНАЯ АЛГЕБРА,http://libmeta.ru/me/article/992_KOMMUTATIVNAJa,70
3,ГЙЫБЕРТОВО ПРОСТРАНСТВО,http://libmeta.ru/me/article/1046_GJYBERTOVO,69
4,МАТЕМАТИЧЕСКАЯ СТАТИСТИКА,http://libmeta.ru/me/article/590_MATEMATIChESKAJa,67
5,АЈГЕБРАИЧЕСКАЯ КРИВАЯ,http://libmeta.ru/me/article/109_AЈGEBRAIChESKAJa,63
6,труТОПОЛОГИЧЕСКАЯ ГРУППА,http://libmeta.ru/me/article/422_truTOPOLOGICh...,63
7,ПРЕДСТАВЛЕНИЕ ТОПОЛОГИЧЕСКОИ,http://libmeta.ru/me/article/619_PREDSTAVLENIE,62
8,ОРТОГОНАЛЬНАЯ ГРУППА,http://libmeta.ru/me/article/68_ORTOGONAL'NAJa,62
9,АСТРОФИЗИКИ МАТЕМАТИЧЕСКИЕ ЗАДАЧИ,http://libmeta.ru/me/article/313_ASTROFIZIKI,59


## 6. Изолированные узлы

Изолированный узел — статья с нулевой степенью (ни входящих, ни исходящих связей по учтённым объектным свойствам).

In [67]:
isolated = [a for a in article_uri_to_label if total_degree.get(a, 0) == 0]
n_isolated = len(isolated)
print(f"Количество изолированных узлов: {n_isolated} ({100 * n_isolated / n_nodes:.2f}%)")
if n_isolated > 0 and n_isolated <= 20:
    display(pd.DataFrame([{"Название": article_uri_to_label.get(a, a)} for a in isolated]))
elif n_isolated > 20:
    print("(Показаны первые 20)")
    display(pd.DataFrame([{"Название": article_uri_to_label.get(a, a)} for a in isolated[:20]]))

Количество изолированных узлов: 1584 (16.23%)
(Показаны первые 20)


,Название
0,ИНВАРИАНТНАЯ РЕГУЛЯРИЗАЦИЯ
1,ИНВАРИАНТНОЕ МНОГООБРАЗИЕ
2,ИНВАРИАНТНОЕ МНОЖЕСТВО
3,ИНВАРИАНТНЫЙ ЗАРЯД В КВАНТОВОЙ ТЕОРИИ ПОЛЯ
4,АНТИСИММЕТРИЧНОЕ ПРОСТРАНСТВО ФОКА
5,ИНВАРИАНТНЫЙ ТОР
6,ИНВЕРСИИ МЕТОД
7,"ИНЕРТНАЯ МАССА, ИНЕРЦИАЛЬНАЯ МАССА"
8,"ИНТЕГРАЛ ПРЕДСТАВЛЕНИЙ, ПРЯМОЙ ИНТЕГРАЛ ПРЕДСТ..."
9,ИНТЕГРАЛЬНАЯ КРИВАЯ


## 7. Плотность графа

Плотность ориентированного графа: $D = \\frac{|E|}{|V|(|V|-1)}$ (доля существующих дуг от максимально возможного числа).

In [68]:
max_edges = n_nodes * (n_nodes - 1) if n_nodes > 1 else 0
density = n_edges / max_edges if max_edges else 0.0
print(f"Плотность (ориентированный граф): {density:.6f}")
print(f"  Рёбер: {n_edges}, макс. возможных: {max_edges}")

Плотность (ориентированный граф): 0.000605
  Рёбер: 57568, макс. возможных: 95208806


## 8. Метрики центральности

Граф для центральности строится **только по связям статей между собой**: ребро A → B, если статья A ссылается на термин статьи B (`hasRelation` на Term). Остальные связи (авторы, публикации, формулы) в этом подграфе не учитываются.

In [69]:
# Орграф статей: ребро (a1, a2) если a1 hasRelation term и a2 hasTerm term
G_articles = nx.DiGraph()
article_set = set(str(a) for a in articles)
for a in articles:
    G_articles.add_node(str(a))

for s, _, term in g.triples((None, SG.hasRelation, None)):
    if (s, RDF.type, SG.Article) not in g:
        continue
    src = str(s)
    for tgt in term_to_articles.get(str(term), []):
        if tgt != src and tgt in article_set:
            G_articles.add_edge(src, tgt)

print(f"Узлов в графе связей статей: {G_articles.number_of_nodes()}")
print(f"Дуг (статья → статья): {G_articles.number_of_edges()}")

Узлов в графе связей статей: 9758
Дуг (статья → статья): 66518


### 8.1 Betweenness Centrality (центральность по посредничеству)

In [70]:
# Для графа без рёбер nx.betweenness_centrality всё равно вернёт словарь с нулями
betweenness = nx.betweenness_centrality(G_articles) if G_articles.number_of_edges() else {n: 0.0 for n in G_articles}
print("Топ-10 по центральности по посредничеству (Betweenness):")
display(top10_df(betweenness, "Betweenness"))

Топ-10 по центральности по посредничеству (Betweenness):


,Название статьи,URI,Betweenness
0,АЛГЕБРА,http://libmeta.ru/me/article/94_ALGEBR,0.011818
1,ФУНКТОР,http://libmeta.ru/me/article/790_FUNKTO,0.011395
2,ПРЕДСТАВЛЕНИЕ,http://libmeta.ru/me/article/3_PREDSTAVLENI,0.010701
3,ПОРЯДОК,http://libmeta.ru/me/article/531_PORJaDO,0.009569
4,ГОМЕОМОРФИЗМ,http://libmeta.ru/me/article/1116_GOMEOMORFIZ,0.007626
5,ПОЛУГРУППА,http://libmeta.ru/me/article/457_POLUGRUPP,0.005996
6,НЕРАВЕНСТВО,http://libmeta.ru/me/article/973_NERAVENSTV,0.005456
7,ЧИСЛО,http://libmeta.ru/me/article/1009_ChISL,0.004957
8,КОМПАКТНОСТЬ,http://libmeta.ru/me/article/1005_KOMPAKTNOST,0.004939
9,ПОЛI0С,http://libmeta.ru/me/article/501_POLI0,0.004858


### 8.2 Eigenvector Centrality (центральность по собственному вектору)

In [71]:
try:
    eigenvector = nx.eigenvector_centrality(G_articles, max_iter=500)
except (nx.PowerIterationFailedConvergence, nx.NetworkXError):
    eigenvector = {n: 0.0 for n in G_articles}
print("Топ-10 по центральности по собственному вектору (Eigenvector):")
display(top10_df(eigenvector, "Eigenvector"))

Топ-10 по центральности по собственному вектору (Eigenvector):


,Название статьи,URI,Eigenvector
0,МНОЖЕСТВО,http://libmeta.ru/me/article/733_MNOZhESTV,0.340007
1,ПРОСТРАНСТВО,http://libmeta.ru/me/article/748_PROSTRANSTV,0.313199
2,ПРОСТРАНСТВО,http://libmeta.ru/me/article/752_PROSTRANSTV,0.313199
3,ТЕОРИЯ,http://libmeta.ru/me/article/381_TEORI,0.279504
4,ФУНКТОР,http://libmeta.ru/me/article/790_FUNKTO,0.223286
5,СИСТЕМА,http://libmeta.ru/me/article/350_SISTEM,0.209984
6,МНОГООБРАЗИЙ,http://libmeta.ru/me/article/438_MNOGOOBRAZI,0.177082
7,ПЕРЕГО́РОДК,http://libmeta.ru/me/article/222_PEREGÓROD,0.152396
8,ПРОИЗВОДНАЯ,http://libmeta.ru/me/article/1045_PROIZVODNA,0.148069
9,КОМПАКТНОСТЬ,http://libmeta.ru/me/article/1005_KOMPAKTNOST,0.129261


### 8.3 PageRank

In [72]:
pagerank = nx.pagerank(G_articles)
print("Топ-10 по PageRank:")
display(top10_df(pagerank, "PageRank"))

Топ-10 по PageRank:


,Название статьи,URI,PageRank
0,МНОЖЕСТВО,http://libmeta.ru/me/article/733_MNOZhESTV,0.021868
1,СИСТЕМА,http://libmeta.ru/me/article/350_SISTEM,0.016727
2,ФУНКТОР,http://libmeta.ru/me/article/790_FUNKTO,0.016543
3,ТЕОРИЯ,http://libmeta.ru/me/article/381_TEORI,0.016528
4,ПРОСТРАНСТВО,http://libmeta.ru/me/article/748_PROSTRANSTV,0.016185
5,ПРОСТРАНСТВО,http://libmeta.ru/me/article/752_PROSTRANSTV,0.016185
6,ФОРМУЛА,http://libmeta.ru/me/article/741_FORMUL,0.010982
7,МНОГООБРАЗИЙ,http://libmeta.ru/me/article/438_MNOGOOBRAZI,0.010536
8,АКСИОМАТИЧЕСКИИ МЕТОД,http://libmeta.ru/me/article/90_AKSIOMATIChESKII,0.007457
9,ПРОИЗВОДНАЯ,http://libmeta.ru/me/article/1045_PROIZVODNA,0.007420


## Дополнение: SPARQL из Python (GraphDB)

Если данные загружены в GraphDB, те же расчёты можно выполнять через SPARQL. Ниже — пример: топ статей по исходящей степени через эндпоинт. Задайте `GRAPHDB_ENDPOINT` или переменную `endpoint`.

In [73]:
# Пример SPARQL: топ-10 статей по исходящей степени (с названием)
SPARQL_TOP_OUT_DEGREE = """
PREFIX sg: <https://scilib.ai/ontology/semantic-graph/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?article ?label (COUNT(?o) AS ?outDegree) {
  ?article a sg:Article ;
          rdfs:label ?label ;
          ?p ?o .
  FILTER (?p IN (sg:hasAuthor, sg:hasTerm, sg:hasThesaurus, sg:refersTo, sg:hasFormula, sg:hasRelation))
} GROUP BY ?article ?label
ORDER BY DESC(?outDegree)
LIMIT 10
"""
# Выполнение (раскомментируйте и укажите endpoint):
# from graph_loader import run_sparql, get_sparql_endpoint
# rows = run_sparql(SPARQL_TOP_OUT_DEGREE, endpoint=get_sparql_endpoint())
# pd.DataFrame(rows)